Libraries Used

In [1]:
import numpy as np
import pandas as pd
import sqlite3
from xml.dom import minidom
import xml.etree.ElementTree as et
from xml.etree.ElementTree import Element
from xml.etree.ElementTree import ElementTree

In [72]:
arr = df.columns.values
def convertToXml(df,filename):
    root = minidom.Document()
    xml = root.createElement("root")
    root.appendChild(xml)
    l = len(df.index)
    s = pd.Series(df.columns.values, name='x')
    for i in range(0,l):
            productChild = root.createElement('product')
            xml.appendChild(productChild)
            row = df.loc[i]
            for j in range(0,4):
                tagname = arr[j]
                tagvalue =str(row[j])
                childOfProduct = root.createElement(tagname)
                childOfProduct.appendChild(root.createTextNode(tagvalue))
                productChild.appendChild(childOfProduct)
                
    xml_str = root.toprettyxml(indent="\t")
    with open(filename+".xml","w") as f:
         f.write(xml_str)
def transferfromXmlToDb(filename,dbfile):
    tree = et.parse(filename+".xml")
    root = tree.getroot()
    conn = sqlite3.connect(dbfile+'.db')
    c = conn.cursor()
    for child in root :
        if child.tag == 'product' :
            list3=[] 
            for attr in child :
#                 if attr.tag == 'a':
#                     a = attr.text
#                 if attr.tag == 'b':
#                     b = attr.text
#                 if attr.tag == 'c':
#                     l = attr.text
#                 if attr.tag == 'd':
#                     d = attr.text
                list3.append(attr.text)
               # print(list3)
        c.execute("insert into " + dbfile + " values (" + ('?,' * len(list3))[:-1] + ")",list3)
            #c.execute("INSERT INTO {} VALUES ('{}','{}','{}','{}')".format(dbfile,a,b,l,d))
        conn.commit()

    conn.close()
def createSchema(dbname):
    conn = sqlite3.connect(dbname+'.db')
    c = conn.cursor()
    query = "Create table {} (".format(dbname);
    length = len(arr)
    i = 0
    for i in  range(length - 1) :
        query = query + arr[i] + ' text,'
        
    query = query + arr[length-1] + ' text)'
    print(query)
    c.execute(query)
#     create query = 
#     c.execute("""CREATE TABLE {}(
#             a text,
#             b text,
#             c text,
#             d text
#             )""".format(dbname))
    conn.commit()
    conn.close()
def checkDb(dbname):
    conn = sqlite3.connect(dbname+'.db')
    c = conn.cursor()
    #c.execute("INSERT INTO employees VALUES ('9','122','11','12')")
    c.execute("SELECT * FROM {}".format(dbname))
    print(c.fetchall())
    conn.commit()
    conn.close()

In [68]:
#df = pd.read_excel('kyphosis.xlsx')
df = pd.read_csv('kyphosis.csv')

Displaying data of excel/csv file

In [69]:
df

,Kyphosis,Age,Number,Start
0,absent,71,3,5
1,absent,158,3,14
2,present,128,4,5
3,absent,2,5,1
4,absent,1,4,15
5,absent,1,2,16
6,absent,61,2,17
7,absent,37,3,16
8,absent,113,2,16
9,present,59,6,12


In [70]:
#convertToXml(df,<xml file name to be created>)
convertToXml(df,"kyphosisCheck")

In [73]:
#createSchema(table name to be created)
createSchema("kyphosisNew")

Create table kyphosisNew (Kyphosis text,Age text,Number text,Start text)


In [75]:
#transferfromXmlToDb(<xml file name>,<table name>)
transferfromXmlToDb("kyphosisCheck","kyphosisNew")

In [83]:
#checkDb("db table name")
checkDb("kyphosisNew")

[('absent', '71', '3', '5'), ('absent', '158', '3', '14'), ('present', '128', '4', '5'), ('absent', '2', '5', '1'), ('absent', '1', '4', '15'), ('absent', '1', '2', '16'), ('absent', '61', '2', '17'), ('absent', '37', '3', '16'), ('absent', '113', '2', '16'), ('present', '59', '6', '12'), ('present', '82', '5', '14'), ('absent', '148', '3', '16'), ('absent', '18', '5', '2'), ('absent', '1', '4', '12'), ('absent', '168', '3', '18'), ('absent', '1', '3', '16'), ('absent', '78', '6', '15'), ('absent', '175', '5', '13'), ('absent', '80', '5', '16'), ('absent', '27', '4', '9'), ('absent', '22', '2', '16'), ('present', '105', '6', '5'), ('present', '96', '3', '12'), ('absent', '131', '2', '3'), ('present', '15', '7', '2'), ('absent', '9', '5', '13'), ('absent', '8', '3', '6'), ('absent', '100', '3', '14'), ('absent', '4', '3', '16'), ('absent', '151', '2', '16'), ('absent', '31', '3', '16'), ('absent', '125', '2', '11'), ('absent', '130', '5', '13'), ('absent', '112', '3', '16'), ('absent', 

In [80]:
df1 = pd.DataFrame(checkDb("kyphosisNew"))

[('absent', '71', '3', '5'), ('absent', '158', '3', '14'), ('present', '128', '4', '5'), ('absent', '2', '5', '1'), ('absent', '1', '4', '15'), ('absent', '1', '2', '16'), ('absent', '61', '2', '17'), ('absent', '37', '3', '16'), ('absent', '113', '2', '16'), ('present', '59', '6', '12'), ('present', '82', '5', '14'), ('absent', '148', '3', '16'), ('absent', '18', '5', '2'), ('absent', '1', '4', '12'), ('absent', '168', '3', '18'), ('absent', '1', '3', '16'), ('absent', '78', '6', '15'), ('absent', '175', '5', '13'), ('absent', '80', '5', '16'), ('absent', '27', '4', '9'), ('absent', '22', '2', '16'), ('present', '105', '6', '5'), ('present', '96', '3', '12'), ('absent', '131', '2', '3'), ('present', '15', '7', '2'), ('absent', '9', '5', '13'), ('absent', '8', '3', '6'), ('absent', '100', '3', '14'), ('absent', '4', '3', '16'), ('absent', '151', '2', '16'), ('absent', '31', '3', '16'), ('absent', '125', '2', '11'), ('absent', '130', '5', '13'), ('absent', '112', '3', '16'), ('absent', 

""
